In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

***Cargando los datasets***

In [2]:
def load_mesas(file):
    mesas = pd.read_csv(file, sep = '|', low_memory = False)
    mesas['CODIGO_CIRCUITO'] = mesas['CODIGO_CIRCUITO'].astype('str')
    mesas['CODIGO_MESA'] = mesas['CODIGO_MESA'].astype('str')

    return mesas
mesas_p = load_mesas('mesas_totales_paso.dsv')
mesas_p.head()

,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,CONTADOR,VALOR
0,1,1001,01001000001,0100100001X,100000000000,VB,3
1,1,1001,01001000001,0100100001X,100000000000,VII,0
2,1,1001,01001000001,0100100001X,100000000000,VN,3
3,1,1001,01001000001,0100100001X,100000000000,VR,0
4,1,1001,01001000001,0100100001X,201000000000,VB,0


In [3]:
votos_p = load_mesas('mesas_totales_agrp_politica_paso.dsv')
votos_p.head()

,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,CODIGO_AGRUPACION,VOTOS_AGRUPACION
0,1,1001,01001000001,0100100001X,100000000000,13,1
1,1,1001,01001000001,0100100001X,100000000000,131,5
2,1,1001,01001000001,0100100001X,100000000000,132,2
3,1,1001,01001000001,0100100001X,100000000000,133,15
4,1,1001,01001000001,0100100001X,100000000000,135,85


In [4]:
agrupaciones_p = pd.read_csv('descripcion_postulaciones_paso.dsv', sep = '|')
agrupaciones_p.head()

,CODIGO_CATEGORIA,NOMBRE_CATEGORIA,CODIGO_AGRUPACION,NOMBRE_AGRUPACION,CODIGO_LISTA,NOMBRE_LISTA
0,801000000000,Diputados Ciudad Autónoma de Buenos Aires,01-504,CONSENSO FEDERAL,000801000000000.01-504.A.000801000000000.01-504,CONSENSO PARA EL FUTURO
1,801000000000,Diputados Ciudad Autónoma de Buenos Aires,13,MOVIMIENTO AL SOCIALISMO,000801000000000.13.A.000801000000000.13,UNIDAD DE LA IZQUIERDA
2,801000000000,Diputados Ciudad Autónoma de Buenos Aires,5,DEMOCRATA CRISTIANO,000801000000000.5.A.000801000000000.5,CIUDAD CELESTE POR LA VIDA Y LA FAMILIA
3,801000000000,Diputados Ciudad Autónoma de Buenos Aires,87,UNITE POR LA LIBERTAD Y LA DIGNIDAD,000801000000000.87.B.000801000000000.87,LISTA DIGNIDAD
4,801000000000,Diputados Ciudad Autónoma de Buenos Aires,87,UNITE POR LA LIBERTAD Y LA DIGNIDAD,000801000000000.87.A.000801000000000.87,OLA CELESTE


***FIltrado del dataset***

In [5]:
mesas_paso = mesas_p.copy()
votos_paso = votos_p.copy()
agrupaciones_paso = agrupaciones_p.copy()

In [6]:
def filter_codigo_categoria(df):
    
    df = df[df.CODIGO_CATEGORIA == 100000000000]
    df = df.reset_index(drop = True)
    return df
    
mesas_paso = filter_codigo_categoria(mesas_paso)
votos_paso = filter_codigo_categoria(votos_paso)
agrupaciones_paso = filter_codigo_categoria(agrupaciones_paso)    

In [7]:
def voto_blanco(df, string, new_column_name):
    df = df[df.CONTADOR == string]
    df = df.rename(columns = {'VALOR': new_column_name})
    df = df.drop(columns = ['CONTADOR'])    
    df = df.reset_index(drop = True)
    return df

In [8]:
mesas_paso = voto_blanco(mesas_paso, 'VB', 'votos_blanco_paso')
mesas_paso

,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,votos_blanco_paso
0,1,1001,01001000001,0100100001X,100000000000,3
1,1,1001,01001000001,0100100002X,100000000000,0
2,1,1001,01001000001,0100100003X,100000000000,6
3,1,1001,01001000001,0100100004X,100000000000,9
4,1,1001,01001000001,0100100005X,100000000000,10
...,...,...,...,...,...,...
98823,24,24005,24005000021,2400500480X,100000000000,5
98824,24,24005,24005000021,2400500481X,100000000000,1
98825,24,24005,24005000021,2400500482X,100000000000,7
98826,24,24005,24005000021,2400500483X,100000000000,12


In [9]:
print(len(mesas_paso.CODIGO_MESA.unique()))
print(len(votos_paso.CODIGO_MESA.unique()))

98828
98834


***Cambiando los códigos de las agrupaciones por sus respectivos nombres***

In [10]:
def agrupacion_codigo_nombre(df):
    
    df = df[['CODIGO_AGRUPACION', 'NOMBRE_AGRUPACION']]
    df = pd.DataFrame(df.groupby(['CODIGO_AGRUPACION','NOMBRE_AGRUPACION']).size())
    df = df.reset_index()
    df = df.iloc[:,:2]
    return df

In [11]:
df_agrp_codigos_paso = agrupacion_codigo_nombre(agrupaciones_paso)
df_agrp_codigos_paso

,CODIGO_AGRUPACION,NOMBRE_AGRUPACION
0,13,MOVIMIENTO AL SOCIALISMO
1,131,FRENTE NOS
2,132,FRENTE PATRIOTA
3,133,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD
4,135,JUNTOS POR EL CAMBIO
5,136,FRENTE DE TODOS
6,137,CONSENSO FEDERAL
7,36,PARTIDO AUTONOMISTA
8,57,MOVIMIENTO DE ACCION VECINAL
9,87,UNITE POR LA LIBERTAD Y LA DIGNIDAD


In [12]:
votos_paso = pd.merge(votos_paso, df_agrp_codigos_paso, on = 'CODIGO_AGRUPACION', how = 'right')
votos_paso.head()

,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,CODIGO_AGRUPACION,VOTOS_AGRUPACION,NOMBRE_AGRUPACION
0,1,1001,01001000001,0100100001X,100000000000,13,1,MOVIMIENTO AL SOCIALISMO
1,1,1001,01001000001,0100100002X,100000000000,13,6,MOVIMIENTO AL SOCIALISMO
2,1,1001,01001000001,0100100003X,100000000000,13,4,MOVIMIENTO AL SOCIALISMO
3,1,1001,01001000001,0100100004X,100000000000,13,1,MOVIMIENTO AL SOCIALISMO
4,1,1001,01001000001,0100100005X,100000000000,13,5,MOVIMIENTO AL SOCIALISMO


***Agrupado los datos por agrupación***

In [13]:
def agrupar_datos(df_v, nombre_agrupacion, nombre_columna):
    
    df = df_v[df_v.NOMBRE_AGRUPACION == nombre_agrupacion]
    df = df.rename(columns={'VOTOS_AGRUPACION':nombre_columna})
    df = df.reset_index(drop=True)
    df = df[['CODIGO_MESA', nombre_columna]]
    
    return df

In [14]:
df_consenso_federal = agrupar_datos(votos_paso, 'CONSENSO FEDERAL', 'Votos_consenso_federal_paso')
df_mas = agrupar_datos(votos_paso, 'MOVIMIENTO AL SOCIALISMO', 'Votos_MAS_paso')
df_fit = agrupar_datos(votos_paso, 'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD', 'Votos_fit_paso')
df_todos = agrupar_datos(votos_paso, 'FRENTE DE TODOS', 'Votos_Frente_todos_paso')
df_nos = agrupar_datos(votos_paso, 'FRENTE NOS', 'Votos_NOS_paso')
df_frente_patriota = agrupar_datos(votos_paso, 'FRENTE PATRIOTA', 'Votos_frente_patriota_paso')
df_juntos_cambio = agrupar_datos(votos_paso, 'JUNTOS POR EL CAMBIO', 'Votos_juntos_cambio_paso')
df_accion_vecinal = agrupar_datos(votos_paso, 'MOVIMIENTO DE ACCION VECINAL', 'Votos_accion_vecinal_paso')
df_autonomista = agrupar_datos(votos_paso, 'PARTIDO AUTONOMISTA', 'Votos_autonomista_paso')
df_unite = agrupar_datos(votos_paso, 'UNITE POR LA LIBERTAD Y LA DIGNIDAD', 'Votos_unite_paso')

In [15]:
from functools import reduce

dataframes = [df_consenso_federal, df_mas, df_fit, df_todos, df_nos, df_frente_patriota, df_juntos_cambio, df_accion_vecinal,
             df_autonomista, df_unite]

df_paso = reduce(lambda  left,right: pd.merge(left,right,on=['CODIGO_MESA'],
                                            how='outer'), dataframes)
df_paso

,CODIGO_MESA,Votos_consenso_federal_paso,Votos_MAS_paso,Votos_fit_paso,Votos_Frente_todos_paso,Votos_NOS_paso,Votos_frente_patriota_paso,Votos_juntos_cambio_paso,Votos_accion_vecinal_paso,Votos_autonomista_paso,Votos_unite_paso
0,0100100001X,20,1,15,101,5,2,85,1,0,4
1,0100100002X,14,6,9,99,8,1,89,0,0,9
2,0100100003X,14,4,15,106,3,0,94,0,0,6
3,0100100004X,21,1,9,111,3,0,80,0,0,16
4,0100100005X,19,5,10,109,2,0,93,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...
98829,2400500480X,18,0,3,102,31,1,54,0,1,2
98830,2400500481X,24,1,4,118,10,1,29,0,0,3
98831,2400500482X,15,1,5,116,15,1,31,0,1,2
98832,2400500483X,14,0,2,126,14,3,34,0,0,0


***Combinamos el dataset con votos en blanco con el de votos a agrupaciones***

In [16]:
mesas_paso = mesas_paso[['CODIGO_MESA','votos_blanco_paso']]

In [17]:
df_votos_paso = pd.merge(mesas_paso, df_paso, on = 'CODIGO_MESA', how = 'right' )

In [18]:
df_votos_paso.isnull().sum()

CODIGO_MESA                    0
votos_blanco_paso              6
Votos_consenso_federal_paso    0
Votos_MAS_paso                 0
Votos_fit_paso                 0
Votos_Frente_todos_paso        0
Votos_NOS_paso                 0
Votos_frente_patriota_paso     0
Votos_juntos_cambio_paso       0
Votos_accion_vecinal_paso      0
Votos_autonomista_paso         0
Votos_unite_paso               0
dtype: int64

In [19]:
df_votos_paso.dropna(inplace = True)

***Agregamos una columna con el total de votos afirmativos (votos a agrupaciones) y el total de votos validos (votos a agrupaciones + votos en blanco)***

In [20]:
df_votos_paso['Total_votos_afrimativos'] = df_votos_paso.iloc[:,2:].sum(axis=1)
df_votos_paso['Total_votos_validos'] = df_votos_paso.iloc[:,1:12].sum(axis=1)

In [21]:
df_votos_paso.head()

,CODIGO_MESA,votos_blanco_paso,Votos_consenso_federal_paso,Votos_MAS_paso,Votos_fit_paso,Votos_Frente_todos_paso,Votos_NOS_paso,Votos_frente_patriota_paso,Votos_juntos_cambio_paso,Votos_accion_vecinal_paso,Votos_autonomista_paso,Votos_unite_paso,Total_votos_afrimativos,Total_votos_validos
0,0100100001X,3.0,20,1,15,101,5,2,85,1,0,4,234,237.0
1,0100100002X,0.0,14,6,9,99,8,1,89,0,0,9,235,235.0
2,0100100003X,6.0,14,4,15,106,3,0,94,0,0,6,242,248.0
3,0100100004X,9.0,21,1,9,111,3,0,80,0,0,16,241,250.0
4,0100100005X,10.0,19,5,10,109,2,0,93,0,0,10,248,258.0


***Calculamos los porcentajes obtenidos por cada agrupación y en blanco (se calcula dividiendo el total de votos por mesa emitidos a cada agrupación divido el total de votos válidos por mesa***

In [22]:
df_votos_paso.columns

Index(['CODIGO_MESA', 'votos_blanco_paso', 'Votos_consenso_federal_paso',
       'Votos_MAS_paso', 'Votos_fit_paso', 'Votos_Frente_todos_paso',
       'Votos_NOS_paso', 'Votos_frente_patriota_paso',
       'Votos_juntos_cambio_paso', 'Votos_accion_vecinal_paso',
       'Votos_autonomista_paso', 'Votos_unite_paso', 'Total_votos_afrimativos',
       'Total_votos_validos'],
      dtype='object')

In [23]:
porcentaje = lambda x,y: x/y

df_votos_paso['por_consenso_federal_paso'] = porcentaje(df_votos_paso['Votos_consenso_federal_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_fit_paso'] = porcentaje(df_votos_paso['Votos_fit_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_MAS_paso'] = porcentaje(df_votos_paso['Votos_MAS_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_Frente_todos_paso'] = porcentaje(df_votos_paso['Votos_Frente_todos_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_frente_patriota_paso'] = porcentaje(df_votos_paso['Votos_frente_patriota_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_juntos_cambio_paso'] = porcentaje(df_votos_paso['Votos_juntos_cambio_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_accion_vecinal_paso'] = porcentaje(df_votos_paso['Votos_accion_vecinal_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_autonomista_paso'] = porcentaje(df_votos_paso['Votos_autonomista_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_unite_paso'] = porcentaje(df_votos_paso['Votos_unite_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_NOS_paso'] = porcentaje(df_votos_paso['Votos_NOS_paso'],df_votos_paso['Total_votos_validos'])
df_votos_paso['por_votos_blanco_paso'] = porcentaje(df_votos_paso['votos_blanco_paso'],df_votos_paso['Total_votos_validos'])

****Ahora que ya tenemos listo el dataset de las elecciones PASO, es necesario hacer el mismo proceso pero para
el dataset de las elecciones generales****

***Cargamos los datos***

In [26]:
def load_df(file):
    
    generales = pd.read_csv(file, sep = '|', low_memory = False)
    generales['CODIGO_CIRCUITO'] = generales['CODIGO_CIRCUITO'].astype('str')
    generales['CODIGO_MESA'] = generales['CODIGO_MESA'].astype('str')
    
    return generales

mesas_g = load_df('mesas_totales.dsv')
votos_g = load_df('mesas_agrp_politicas.dsv')
agrupaciones_g =  pd.read_csv('descripcion_postulaciones.dsv',sep='|')

In [27]:
mesas_generales = mesas_g.copy()
votos_generales = votos_g.copy()
agrupaciones_generales = agrupaciones_g.copy()

***Filtramos los datasets***

In [28]:
mesas_generales = filter_codigo_categoria(mesas_generales)
votos_generales = filter_codigo_categoria(votos_generales)
agrupaciones_generales = filter_codigo_categoria(agrupaciones_generales)

In [29]:
def errores_de_carga(df):
    
    mesas_sin_electores = df[df.CONTADOR != 'Total de electores de mesas computadas']
    mesas_sin_electores_agrupada = pd.DataFrame((mesas_sin_electores.groupby(['CODIGO_MESA'])['VALOR'].sum()))
    mesas_sin_electores_agrupada.reset_index(inplace = True)
    
    mesas_electores = df[df.CONTADOR == 'Total de electores de mesas computadas']
    mesas_electores = mesas_electores[['CODIGO_MESA', 'VALOR']]
    mesas_electores.reset_index(drop = True, inplace = True)
    
    mesas_sin_electores_agrupada.rename(columns = {'VALOR': 'Total_votos'}, inplace = True)
    mesas_electores.rename(columns = {'VALOR': 'Total_electores'}, inplace = True)
    
    mesas_comparacion = pd.merge(mesas_sin_electores_agrupada,mesas_electores, on = 'CODIGO_MESA', how = 'right')
    
    mesas_comparacion = mesas_comparacion.drop(mesas_comparacion[(mesas_comparacion.Total_votos > mesas_comparacion.Total_electores)].index)

    mesas_comparacion.reset_index(drop = True, inplace = True)
    
    return mesas_comparacion
    
mesas_sin_errores_carga = errores_de_carga(mesas_generales)

In [30]:
# mesas_sin_errores_carga

In [31]:
mesas_generales = mesas_generales[mesas_generales.CONTADOR == 'Voto blanco']
mesas_generales = mesas_generales.reset_index(drop=True)
mesas_generales = mesas_generales.drop(columns = ['CONTADOR'])
mesas_generales = mesas_generales.rename(columns = {'VALOR':'votos_blanco_gen'})

In [32]:
mesas_generales = mesas_generales[['CODIGO_MESA', 'votos_blanco_gen']]

***Nos vamos a quedar solamente con aquellas mesas que no tengan errores de carga (votos no sean mayores a electores)***

In [33]:
mesas_sin_errores_carga = mesas_sin_errores_carga[['CODIGO_MESA', 'Total_electores']]
    
mesas_generales = pd.merge(mesas_sin_errores_carga, mesas_generales, on = 'CODIGO_MESA', how = 'right')

mesas_generales.dropna(inplace = True)

,CODIGO_MESA,Total_electores,votos_blanco_gen
0,0100100001X,272.0,5
1,0100100002X,250.0,4
2,0100100003X,261.0,1
3,0100100004X,266.0,5
4,0100100005X,269.0,2
...,...,...,...
97277,2400500479X,NaN,9
97278,2400500480X,NaN,5
97279,2400500481X,NaN,5
97280,2400500482X,NaN,5


In [34]:
mesas_generales.dropna(inplace = True)


In [35]:
df_agrp_codigos_gen = agrupacion_codigo_nombre(agrupaciones_generales)
df_agrp_codigos_gen

,CODIGO_AGRUPACION,NOMBRE_AGRUPACION
0,131,FRENTE NOS
1,133,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD
2,135,JUNTOS POR EL CAMBIO
3,136,FRENTE DE TODOS
4,137,CONSENSO FEDERAL
5,87,UNITE POR LA LIBERTAD Y LA DIGNIDAD


In [36]:
votos_generales = pd.merge(votos_generales, df_agrp_codigos_gen, on = 'CODIGO_AGRUPACION', how = 'right')


***Ahora pasamos a agregar a cada agrupación como una columna del dataset***

In [37]:
votos_generales.NOMBRE_AGRUPACION.unique()

array(['FRENTE NOS', 'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD',
       'JUNTOS POR EL CAMBIO', 'FRENTE DE TODOS', 'CONSENSO FEDERAL',
       'UNITE POR LA LIBERTAD Y LA DIGNIDAD'], dtype=object)

In [38]:
df_consenso_federal_gen = agrupar_datos(votos_generales, 'CONSENSO FEDERAL', 'Votos_consenso_federal_gen')
df_fit_gen = agrupar_datos(votos_generales, 'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD', 'Votos_fit_gen')
df_todos_gen = agrupar_datos(votos_generales, 'FRENTE DE TODOS', 'Votos_Frente_todos_gen')
df_nos_gen = agrupar_datos(votos_generales, 'FRENTE NOS', 'Votos_NOS_gen')
df_juntos_cambio_gen = agrupar_datos(votos_generales, 'JUNTOS POR EL CAMBIO', 'Votos_juntos_cambio_gen')
df_unite_gen = agrupar_datos(votos_generales, 'UNITE POR LA LIBERTAD Y LA DIGNIDAD', 'Votos_unite_gen')

In [39]:
data_frames = [df_consenso_federal_gen, df_fit_gen, df_todos_gen, df_nos_gen, df_juntos_cambio_gen, df_unite_gen]

In [40]:
df = reduce(lambda  left,right: pd.merge(left,right,on=['CODIGO_MESA'],
                                            how='right'), data_frames)


In [41]:
df_generales = pd.merge(mesas_generales, df, how = 'right', on = 'CODIGO_MESA')

In [42]:
df_generales.dropna(inplace = True)

In [43]:
df_generales['Total_votos_afirmativos_gen'] = df_generales.iloc[:,3:9].sum(axis = 1)
df_generales['Total_votos_validos_gen'] =df_generales.iloc[:,2:9].sum(axis = 1)

***Calculamos los porcentajes que sacó cada agrupación***

In [45]:
porcentaje = lambda x,y: x/y

df_generales.loc[:,'por_consenso_federal_gen'] = porcentaje(df_generales['Votos_consenso_federal_gen'],df_generales['Total_votos_validos_gen'])
df_generales.loc[:,'por_fit_gen'] = porcentaje(df_generales['Votos_fit_gen'],df_generales['Total_votos_validos_gen'])
df_generales.loc[:,'por_juntos_cambio_gen'] = porcentaje(df_generales['Votos_juntos_cambio_gen'],df_generales['Total_votos_validos_gen'])
df_generales.loc[:,'por_frente_todos_gen'] = porcentaje(df_generales['Votos_Frente_todos_gen'],df_generales['Total_votos_validos_gen'])
df_generales.loc[:,'por_NOS_gen'] = porcentaje(df_generales['Votos_NOS_gen'],df_generales['Total_votos_validos_gen'])
df_generales.loc[:,'por_Unite_gen'] = porcentaje(df_generales['Votos_unite_gen'],df_generales['Total_votos_validos_gen'])
df_generales.loc[:,'por_blancos_gen'] = porcentaje(df_generales['votos_blanco_gen'],df_generales['Total_votos_validos_gen'])



In [46]:
print(len(df_votos_paso)) 
print(len(df_generales))

98828
60903


In [49]:
df_limpio = df_generales.merge(df_votos_paso, on = 'CODIGO_MESA',
                       how = 'right', sort = False)

***Creamos una columna que tenga el total de votos válidos por mesa, luego eliminamos aquellas cuyo valor sea igual a 0***

In [50]:
def total_votos_validos_mesa(df):
    
    df.loc[:,'Total_votos_validos_mesa'] = df['Total_votos_validos_gen'] + df['Total_votos_validos']
    df = df[df.Total_votos_validos_mesa > 0]
    df.dropna(inplace = True)
    return df

df_limpio = total_votos_validos_mesa(df_limpio)
df_limpio.shape

C:\Users\Administrador\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


(59085, 43)

In [51]:
df_limpio.to_csv(r'C:\Users\Administrador\Desktop\Carrera de Data Science\DS-Cor\Transferencia votos\dataframe_limpio2323.csv')